## Proyecto 1 

Nicolas Arango - 202220342
Mateo Rincon - 202221402
Amalia Carbonell - 202122079 



1. Entendimiento y preparación de los datos (a nivel de código y análisis).
2. Modelado y evaluación.

### Entendimiento y preparación de los datos (a nivel de código y análisis)

In [1]:
import numpy as np
import pandas as pd

from joblib import dump, load

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import scipy.stats as stats

In [42]:
# Carga de Datos
#noticias_df = pd.read_csv("./fake_df/fake_news_test.csv", sep=";", usecols=['ID','Titulo', "Descripcion", "Fecha"])
test_df=pd.read_csv('./data/fake_news_test.csv', sep=';', encoding = 'utf-8')
fake_df = pd.read_csv('./data/fake_news_spanish.csv', sep=';', encoding = 'utf-8')

In [43]:
fake_df.shape


(57063, 5)

In [44]:
fake_df.dtypes

ID                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [45]:
fake_df.head()



,ID,Label,Titulo,Descripcion,Fecha
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018


### Describir los Datos

Se hace una descripción más detallada de nuestros datos.Es un paso es muy importante para poder determinar problemas de calidad de datos. Además, nos dará información que será útil en la fase de preparación de datos.

In [46]:
fake_df.describe()

,Label
count,57063.000000
mean,0.581077
std,0.493387
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [47]:
fake_df.sample(5)

,ID,Label,Titulo,Descripcion,Fecha
27861,ID,1,El TS sentencia que TV3 vulneró la neutralidad...,El tribunal considera que el documental proyec...,26/02/2021
56308,ID,0,"Rocío Monasterio, el juez condenado que quiso ...","En su huida hacia adelante, acabó grabando has...",14/09/2019
34280,ID,0,Esther López insinúa que no irá en las listas ...,La ministra de Economía afirma que no cree que...,28/02/2019
23994,ID,0,Bélgica acoge una cumbre independentista para ...,El encuentro sirvió para hacer un repaso gener...,03/12/2018
47887,ID,0,"Las elecciones andaluzas, marcadas por la ince...",La favorita a vencer en las elecciones andaluz...,01/12/2018


In [48]:
features = ['Titulo', 'Descripcion', 'Fecha']

In [50]:
#Revisión del porcentaje de valores nulos (vacíos) en las características seleccionadas para la regresión.
fake_df[['ID'] + features].isnull().sum() / fake_df.shape[0]

KeyError: "['ID', 'Titulo', 'Descripcion'] not in index"

In [ ]:
fake_df = fake_df.dropna(subset=["Titulo"]+features)

In [ ]:
fake_df.shape
total_rows = fake_df.shape[0]
total_rows

999

In [ ]:
fake_df.loc[fake_df.duplicated(subset=features, keep=False)].head(3)

,ID,Titulo,Descripcion,Fecha
113,ID,El gran temor de PP y Cs de cara al 4M: la mov...,El principal miedo que se ha instalado en el P...,22/04/2021
535,ID,El gran temor de PP y Cs de cara al 4M: la mov...,El principal miedo que se ha instalado en el P...,22/04/2021


In [ ]:
duplicated_rows = fake_df.loc[fake_df.duplicated(subset=features, keep=False)].shape[0]
duplicated_rows

2

In [22]:
print(f"Duplicates: {(duplicated_rows/total_rows)*100:.4f}%")

Duplicates: 0.2002%


In [ ]:
fake_df.drop_duplicates(subset=features, inplace=True)
fake_df.drop_duplicates(subset=features+["Titulo"], inplace=True)

In [ ]:
fake_df.shape

(998, 4)

In [ ]:
titulosCortos = fake_df['Titulo'].str.len() < 25
print(f"Titulos cortos: {titulosCortos.sum()}")
fake_df.loc[titulosCortos, 'Titulo']

Titulos cortos: 1


249    El más listo de la clase
Name: Titulo, dtype: object